In [ ]:
# Activate intractive functionality of matplotlib
%matplotlib ipympl
import numpy as np
import pandas as pd
import h5py
from tqdm import tqdm
import matplotlib.pyplot as plt
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
    
from pyccapt.calibration.data_tools import data_loadcrop
from pyccapt.calibration.data_tools import raw_data_surface_concept
from pyccapt.calibration.calibration.mc_plot import AptHistPlotter
from pyccapt.calibration.data_tools.data_loadcrop import plot_crop_fdm
from pyccapt.calibration.mc import mc_tools
from pyccapt.calibration.reconstructions import sdm, rdf
from pyccapt.calibration.mc import mc_tools

In [ ]:
filename = 'D:/pyccapt/tests/data/data_1642_Aug-30-2023_16-05_Al_test4.h5'
# filename = 'D:/pyccapt/tests/data/1891_Jul-05-2024_12-10_Al_test_50um_1.h5'
# filename = 'C:/Users/mehrp/PycharmProjects/pyccapt/tests/data/data_1642_Aug-30-2023_16-05_Al_test4.h5'
df_dld = data_loadcrop.fetch_dataset_from_dld_grp(filename, extract_mode='dld')

In [ ]:
df_dld

In [ ]:
tof = df_dld['t (ns)'].to_numpy()
tof_plot = AptHistPlotter(tof[tof<1000])
y_hist, x_hist = tof_plot.plot_histogram()

In [ ]:
print(np.max(df_dld['t (ns)'].to_numpy()))
print(np.max(df_dld['x_det (cm)'].to_numpy()))
print(np.min(df_dld['x_det (cm)'].to_numpy()))
print(np.max(df_dld['y_det (cm)'].to_numpy()))
print(np.min(df_dld['y_det (cm)'].to_numpy()))

plot_crop_fdm(df_dld)
# mc = mc_tools.tof2mc(t=np.array(tof_total), t0=0, V=np.array(high_voltage_total), xDet=np.array(det_x_total), yDet=np.array(det_x_total), flightPathLength=110, V_pulse=np.array(pulse_total), mode='voltage')
mc = mc_tools.tof2mc(t=df_dld['t (ns)'].to_numpy(), t0=0, V=df_dld['high_voltage (V)'].to_numpy(), xDet=df_dld['x_det (cm)'].to_numpy(), yDet=df_dld['y_det (cm)'].to_numpy(), flightPathLength=110, V_pulse=np.zeros(len(df_dld)), mode='voltage')
mc_plot = AptHistPlotter(mc[mc<50])
y_hist, x_hist = mc_plot.plot_histogram()

In [ ]:
# extract raw data from hdf file
df_tdc = data_loadcrop.fetch_dataset_from_dld_grp(filename, extract_mode='tdc_sc')

In [ ]:
df_tdc

In [ ]:
channel = df_tdc['channel'].to_numpy()
start_counter = df_tdc['start_counter'].to_numpy()
time_data = df_tdc['time_data'].to_numpy()
high_voltage = df_tdc['high_voltage (V)'].to_numpy()
pulse = df_tdc['pulse'].to_numpy()

In [ ]:
result_4, result_4_invalid, result_3_invalid, result_2_invalid, result_1_invalid, result_other_odd, result_other_even = raw_data_surface_concept.find_consecutive_sequences_seperatly(start_counter, channel, time_data, high_voltage, pulse)

In [ ]:
print(len(result_4))
print(len(result_other_even))
print(len(result_other_odd))

even_ions = 0
odd_ions = 0
ions_4 = 0
for i in range(len(result_4)):         
    ions_4 += result_4[i]['valid_event'].count(True)
for i in range(len(result_other_even)):         
    even_ions += result_other_even[i]['valid_event'].count(True)
for i in range(len(result_other_odd)):         
    odd_ions += result_other_odd[i]['valid_event'].count(True)

print(ions_4)
print(even_ions)
print(odd_ions)
print('maximum possible number of ions:', ions_4 + even_ions + odd_ions)

In [ ]:
max_length_even = 0
max_length_odd = 0

for i in range(len(result_other_even)):         
    if len(result_other_even[i]['channels']) > max_length_even:
        max_length_even = len(result_other_even[i]['channels'])
for i in range(len(result_other_odd)):         
    if len(result_other_odd[i]['channels']) > max_length_odd:
        max_length_odd = len(result_other_odd[i]['channels'])
        
print('The maximum number of channels for even start counter:', max_length_even)
print('The maximum number of channels for odd start counter:', max_length_odd)

In [ ]:
# surface concept tdc specific binning and factors
TOFFACTOR = 27.432 / (1000 * 4)  # 27.432 ps/bin, tof in ns, data is TDC time sum
TOFFACTOR2 = 27.432 / (1000 * 2) 
DETBINS = 4900
BINNINGFAC = 2
XYFACTOR = 80 / DETBINS * BINNINGFAC  # XXX mm/bin
XYBINSHIFT = DETBINS / BINNINGFAC / 2  # to center detector

In [ ]:
index = 1
aa = df_dld.iloc[index]
print(aa)
print(result_4[index])
time_data_tmp = np.array(result_4[index]['time_data'], dtype=np.int64)

In [ ]:
xx_dif = time_data_tmp[1] - time_data_tmp[0]
yy_dif = time_data_tmp[3] - time_data_tmp[2]
print(xx_dif, yy_dif)
xx_dif = -0.5*xx_dif + 1225
yy_dif = -0.5*yy_dif + 1225
print(xx_dif, yy_dif)
time = time_data_tmp[0] + time_data_tmp[1] + time_data_tmp[2] + time_data_tmp[3]
time2 = time_data_tmp[0] + time_data_tmp[1]
time3 = time_data_tmp[2] + time_data_tmp[3]
det_x = (((xx_dif - XYBINSHIFT) * XYFACTOR) * 0.1)
det_y = (((yy_dif - XYBINSHIFT) * XYFACTOR) * 0.1)
tof = time * TOFFACTOR
tof2 = time2 * TOFFACTOR2
tof3 = time3 * TOFFACTOR2
print('det_x:', det_x, 'det_y:', det_y, 'tof:', tof, 'tof(0-1):', tof2, 'tof(2-3):', tof3)

In [ ]:
det_x_4 = []
det_y_4 = []
tof_4 = []
start_counter_4 = []
high_voltage_4 = []
pulse_4 = []
print('length of result_4:', len(result_4))

for ion in tqdm(result_4, desc="Processing", unit="iteration"):
    if ion['valid_event'][0] == True:
        time_data_tmp = np.array(ion['time_data'], dtype=np.int64)
        xx_dif = time_data_tmp[1] - time_data_tmp[0]
        yy_dif = time_data_tmp[3] - time_data_tmp[2]
        time = time_data_tmp[0] + time_data_tmp[1] + time_data_tmp[2] + time_data_tmp[3]
        xx_dif = -0.5*xx_dif + 1225
        yy_dif = -0.5*yy_dif + 1225
        xx_dif = (((xx_dif - XYBINSHIFT) * XYFACTOR) * 0.1)
        yy_dif = (((yy_dif - XYBINSHIFT) * XYFACTOR) * 0.1)
        if abs(xx_dif) <= 4 and abs(yy_dif) <= 4:
            det_x_4.append(xx_dif)
            det_y_4.append(yy_dif)
            tof_4.append(time * TOFFACTOR)
            start_counter_4.append(np.array(ion['start_counter'], dtype=np.int64)[0])
            high_voltage_4.append(ion['high_voltage'])
            pulse_4.append(ion['pulse'])


print(len(det_x_4))
print('loss:', (len(det_x_4) / len(result_4))*100)
    

In [ ]:
fdm = {'x_det (cm)': det_x_4, 'y_det (cm)': det_y_4}
fdm = pd.DataFrame(fdm)

plot_crop_fdm(fdm)
mc = mc_tools.tof2mc(t=np.array(tof_4), t0=0, V=np.array(high_voltage_4), xDet=np.array(det_x_4), yDet=np.array(det_y_4), flightPathLength=110, V_pulse=np.zeros(len(tof_4)), mode='voltage')
mc_plot = AptHistPlotter(mc[mc<50])
y_hist, x_hist = mc_plot.plot_histogram()

In [ ]:
det_x_even = []
det_y_even = []
tof_even = []
start_counter_even = []
high_voltage_even = []
pulse_even = []
ion_count = 0
even_count = 0
true_event = 0
print('length of result_even:', len(result_other_even))
length_result_even_valid = len([x for x in result_other_even if x['length'] == 4 and x['valid_event'] == [True]])
print()
for ion in tqdm(result_other_even, desc="Processing", unit="iteration"):
    time_data_tmp = np.array(ion['time_data'], dtype=np.int64)
    for index, event in enumerate(ion['valid_event']):
        even_count += 1 
        if event == True:
            true_event += 1
            xx_dif = time_data_tmp[index*4 + 1] - time_data_tmp[index*4 + 0]
            yy_dif = time_data_tmp[index*4 + 3] - time_data_tmp[index*4 + 2]
            time = time_data_tmp[index*4 + 0] + time_data_tmp[index*4 + 1] + time_data_tmp[index*4 + 2] + time_data_tmp[index*4 + 3]
            xx_dif = -0.5*xx_dif + 1225
            yy_dif = -0.5*yy_dif + 1225
            xx_dif = (((xx_dif - XYBINSHIFT) * XYFACTOR) * 0.1)
            yy_dif = (((yy_dif - XYBINSHIFT) * XYFACTOR) * 0.1)
            if abs(xx_dif) <= 4 and abs(yy_dif) <= 4:
                det_x_even.append(xx_dif)
                det_y_even.append(yy_dif)
                tof_even.append(time * TOFFACTOR)
                start_counter_even.append(np.array(ion['start_counter'], dtype=np.int64)[0])
                high_voltage_even.append(ion['high_voltage'])
                pulse_even.append(ion['pulse'])
                ion_count += 1
print(len(det_x_even))
print('loss True event-total event:', (true_event / even_count)*100)
print('loss True event-total ions:', (ion_count / true_event)*100)

In [ ]:
fdm = {'x_det (cm)': det_x_even, 'y_det (cm)': det_y_even}
fdm = pd.DataFrame(fdm)

plot_crop_fdm(fdm)
mc = mc_tools.tof2mc(t=np.array(tof_even), t0=0, V=np.array(high_voltage_even), xDet=np.array(det_x_even), yDet=np.array(det_y_even), flightPathLength=110, V_pulse=np.zeros(len(tof_even)), mode='voltage')
mc_plot = AptHistPlotter(mc[mc<50])
y_hist, x_hist = mc_plot.plot_histogram()

In [ ]:
det_x_odd = []
det_y_odd = []
tof_odd = []
start_counter_odd = []
high_voltage_odd = []
pulse_odd = []
ion_count = 0
even_count = 0
true_event = 0
print('length of result_odd:', len(result_other_odd))

for ion in tqdm(result_other_odd, desc="Processing", unit="iteration"):
    time_data_tmp = np.array(ion['time_data'], dtype=np.int64)
    for index, event in enumerate(ion['valid_event']):
        even_count += 1 
        if event == True:
            true_event += 1
            xx_dif = time_data_tmp[index*4 + 1] - time_data_tmp[index*4 + 0]
            yy_dif = time_data_tmp[index*4 + 3] - time_data_tmp[index*4 + 2]
            time = time_data_tmp[index*4 + 0] + time_data_tmp[index*4 + 1] + time_data_tmp[index*4 + 2] + time_data_tmp[index*4 + 3]
            xx_dif = -0.5*xx_dif + 1225
            yy_dif = -0.5*yy_dif + 1225
            xx_dif = (((xx_dif - XYBINSHIFT) * XYFACTOR) * 0.1)
            yy_dif = (((yy_dif - XYBINSHIFT) * XYFACTOR) * 0.1)
            if abs(xx_dif) <= 4 and abs(yy_dif) <= 4:
                det_x_odd.append(xx_dif)
                det_y_odd.append(yy_dif)
                tof_odd.append(time * TOFFACTOR)
                start_counter_odd.append(np.array(ion['start_counter'], dtype=np.int64)[0])
                high_voltage_odd.append(ion['high_voltage'])
                pulse_odd.append(ion['pulse'])
                ion_count += 1

print(len(det_x_odd))
print('loss True event-total event:', (true_event / even_count)*100)
print('loss True event-total ions:', (ion_count / true_event)*100)

In [ ]:
fdm = {'x_det (cm)': det_x_odd, 'y_det (cm)': det_y_odd}
fdm = pd.DataFrame(fdm)

plot_crop_fdm(fdm)
mc = mc_tools.tof2mc(t=np.array(tof_odd), t0=0, V=np.array(high_voltage_odd), xDet=np.array(det_x_odd), yDet=np.array(det_y_odd), flightPathLength=110, V_pulse=np.zeros(len(tof_odd)), mode='voltage')
mc_plot = AptHistPlotter(mc[mc<50])
y_hist, x_hist = mc_plot.plot_histogram()

In [ ]:
count = 0
for x in result_4:
    for evn in x['valid_event']:
        if evn == True:
            count += 1
for x in result_other_even:
    for evn in x['valid_event']:
        if evn == True:
            count += 1
for x in result_other_odd:
    for evn in x['valid_event']:
        if evn == True:
            count += 1
print(count)

In [ ]:
det_x_total_s = det_x_4 + det_x_even + det_x_odd
det_y_total_s = det_y_4 + det_y_even + det_y_odd
tof_total_s = tof_4 + tof_even + tof_odd
start_counter_total_s = start_counter_4 + start_counter_even + start_counter_odd
high_voltage_total_s = high_voltage_4 + high_voltage_even + high_voltage_odd
pulse_total_s = pulse_4 + pulse_even + pulse_odd

print(len(det_x_total_s))


fdm = {'x_det (cm)': det_x_total_s, 'y_det (cm)': det_y_total_s}
fdm = pd.DataFrame(fdm)
plot_crop_fdm(fdm)
mc = mc_tools.tof2mc(t=np.array(tof_total_s), t0=0, V=np.array(high_voltage_total_s), xDet=np.array(det_x_total_s), yDet=np.array(det_y_total_s), flightPathLength=110, V_pulse=np.zeros(len(det_x_total_s)), mode='voltage')
mc_plot = AptHistPlotter(mc[mc<50])
y_hist, x_hist = mc_plot.plot_histogram()

In [ ]:
result_other_even[2]

In [ ]:
result_other_odd[16]

In [ ]:
result_total = raw_data_surface_concept.find_consecutive_sequences(start_counter, channel, time_data, high_voltage, pulse, print_stats=True)

In [ ]:
ch_1_invalid = 0
ch_2_invalid = 0
ch_2_tof_valid = 0
ch_3_invalid = 0
ch_3_tof_valid = 0
ch_4_invalid = 0
ch_4_tof_valid = 0


channel_lengths = {i: 0 for i in range(1, 501)}
channel_lengths_invalid = {i: 0 for i in range(1, 501)}
channel_lengths_event_valid = {i: 0 for i in range(1, 501)}
channel_lengths_tof_valid = {i: 0 for i in range(1, 501)}

# Count the occurrences of each length
for x in result_total:
    length = len(x['channels'])
    channel_lengths[length] += 1

for x in result_total:
    length = len(x['channels'])
    for i in range(len(x['valid_event'])):
        if x['valid_event'][i] == True:
            channel_lengths_event_valid[length] += 1
        elif x['valid_event'][i] == False:
            if (i+1)*4 < length:
                chs = x['channels'][i*4:(i+1)*4]
            else:
                chs = x['channels'][i*4:]
            if len(chs) > 2:
                if 0 in chs and 1 in chs:
                    channel_lengths_tof_valid[length] += 1
                elif 0 in chs and 1 in chs:
                    channel_lengths_tof_valid[length] += 1
                else:
                    channel_lengths_invalid[length] += 1
            else:
                channel_lengths_invalid[length] += 1

In [ ]:
# Sort the dictionary by keys (channel lengths)
# sorted_lengths = sorted(channel_lengths.items())
# Extract the sorted lengths and counts
# lengths_all, counts_all = zip(*sorted_lengths)
bins = np.arange(1, 500)
channel_lengths_arr = np.array(list(channel_lengths.values()))
channel_lengths_event_valid_arr = np.array(list(channel_lengths_event_valid.values()))
channel_lengths_tof_valid_arr = np.array(list(channel_lengths_tof_valid.values()))
channel_lengths_invalid_arr = np.array(list(channel_lengths_invalid.values()))

index = 20
channel_lengths_arr = channel_lengths_arr[:index]
channel_lengths_event_valid_arr = channel_lengths_event_valid_arr[:index]
channel_lengths_tof_valid_arr = channel_lengths_tof_valid_arr[:index]
channel_lengths_invalid_arr = channel_lengths_invalid_arr[:index]

# Define the width of the bars
width = 0.2
fig, ax = plt.subplots(figsize=(7,2.5))
# Plot bar charts for all arrays
plt.bar(bins[:index], channel_lengths_arr, width=width*2, label='Frequency', alpha=0.5, color='gray')
plt.bar(bins[:index] -  0.5 * width, channel_lengths_tof_valid_arr, width=width, label='2 DLTS', color='orange')
plt.bar(bins[:index] + 0.5 * width, channel_lengths_event_valid_arr, width=width, label='4 DLTS', color='blue')
# plt.bar(bins[:index] - width, channel_lengths_invalid_arr, width=width, label='Not recoverable', color='red')

ax.set_xlabel('Number of Delayline Time Stamps per Pulse', fontsize=10)
plt.xticks(bins[:index], [str(i) for i in range(1, index+1)])
ax.set_ylabel('Count', fontsize=10)
ax.set_yscale('log')
plt.legend(loc='upper right')
plt.tight_layout()
from matplotlib import rcParams
rcParams['svg.fonttype'] = 'none'
plt.savefig("raw_stat.svg", format="svg", dpi=600)
plt.savefig("raw_stat.png", format="png", dpi=600)
plt.show()

for i in range(len(channel_lengths_arr)):
    print('For number:', i+1, '- number of assurance DLETS per pulse:', channel_lengths_arr[i],'- number of possible events:', channel_lengths_arr[i]*(int(i/4)+1),' - which consist of 4 DLETS + 2 DLTS +  not rec:', channel_lengths_event_valid_arr[i] + channel_lengths_tof_valid_arr[i] + channel_lengths_invalid_arr[i] )

In [ ]:

tof_range_1 = 0
tof_range_2 = 0
tof_range_3 = 0
noise_two_d = 0 # whitin the detector area but noise
not_recovable = 0
tof_range_1_one_d = 0
tof_range_2_one_d = 0
tof_range_3_one_d = 0
noise_one_d = 0 # whitin the detector area but noise
not_recovable_two_d_one_end = 0
not_recovable_out_of_detector = 0
num_occurrence = 0

tof_two_d = []
tof_one_d = []
v_dc_two_d = []
v_dc_one_d = []
v_p__two_d = []
v_p__one_d = []
x_det_two_d = []
x_det_one_d = []
y_det_two_d = []
y_det_one_d = []

flag_ch_0_and_1 = False
flag_ch_2_and_3 = False
num_ch_0_and_1 = 0
num_ch_2_and_3 = 0

# for i, x in enumerate(result_total):
for i, x in enumerate(tqdm(result_total)):
    time_data_tmp = np.array(x['time_data'], dtype=np.int64)
    channel_data_tmp = np.array(x['channels'], dtype=np.int64)
    # if i == 3000:
    #     print(i)
    #     break
    for j in range(len(x['valid_event'])):
        if x['valid_event'][j] == True:
            time = time_data_tmp[j * 4 + 0] + time_data_tmp[j * 4 + 1] + time_data_tmp[j * 4 + 2] + time_data_tmp[
                j * 4 + 3]
            time_1 = time * TOFFACTOR
            xx_dif = time_data_tmp[j * 4 + 1] - time_data_tmp[j * 4 + 0]
            yy_dif = time_data_tmp[j * 4 + 3] - time_data_tmp[j * 4 + 2]
            xx_dif = -0.5 * xx_dif + 1225
            yy_dif = -0.5 * yy_dif + 1225
            xx_dif = (((xx_dif - XYBINSHIFT) * XYFACTOR) * 0.1)
            yy_dif = (((yy_dif - XYBINSHIFT) * XYFACTOR) * 0.1)
            if abs(xx_dif) <= 4 and abs(yy_dif) <= 4:
                if 135 < time_1 < 159:
                    tof_range_1 += 1
                elif 392 < time_1 < 463:
                    tof_range_2 += 1
                elif 535 < time_1 < 646:
                    tof_range_3 += 1
                else:
                    noise_two_d += 1
                tof_two_d.append(time_1)
                v_dc_two_d.append(x['high_voltage'])
                x_det_two_d.append(xx_dif)
                y_det_two_d.append(yy_dif)
                v_p__two_d.append(x['pulse'])
                                
            else:
                not_recovable_out_of_detector += 1
            continue

        elif x['valid_event'][j] == False:
            if len(channel_data_tmp[j * 4:]) > 4:
                upper_side = 4
            else:
                upper_side = len(channel_data_tmp[j * 4:])
            channel_data_tmp_2 = channel_data_tmp[j * 4:j * 4 + upper_side]
            time_data_tmp_2 = time_data_tmp[j * 4:j * 4 + upper_side]
            if len(channel_data_tmp[j * 4:j * 4 + upper_side]) == 1:
                not_recovable_two_d_one_end += 1
                continue
            elif len(channel_data_tmp[j * 4:j * 4 + upper_side]) == 2:
                if (channel_data_tmp_2[-2] == 0 and channel_data_tmp_2[-1] == 1):
                    time = time_data_tmp_2[0] + time_data_tmp_2[1]
                    time_1 = time * TOFFACTOR2
                    xx_dif = time_data_tmp_2[1] - time_data_tmp_2[0]
                    xx_dif = -0.5 * xx_dif + 1225
                    xx_dif = (((xx_dif - XYBINSHIFT) * XYFACTOR) * 0.1)
                    tof_one_d.append(time_1)
                    v_dc_one_d.append(x['high_voltage'])
                    x_det_one_d.append(xx_dif)
                    y_det_one_d.append(0)
                    v_p__one_d.append(x['pulse'])
                    if abs(xx_dif) <= 4:
                        flag_time = True
                        flag_ch_0_and_1 = True
                    else:
                        noise_one_d += 1
                        continue
                elif (channel_data_tmp_2[-2] == 2 and channel_data_tmp_2[-1] == 3):
                    time = time_data_tmp_2[0] + time_data_tmp_2[1]
                    time_1 = time * TOFFACTOR2
                    yy_dif = time_data_tmp_2[1] - time_data_tmp_2[0]
                    yy_dif = -0.5 * yy_dif + 1225
                    yy_dif = (((yy_dif - XYBINSHIFT) * XYFACTOR) * 0.1)
                    tof_one_d.append(time_1)
                    v_dc_one_d.append(x['high_voltage'])
                    x_det_one_d.append(0)
                    y_det_one_d.append(yy_dif)
                    v_p__one_d.append(x['pulse'])
                    if abs(yy_dif) <= 4:
                        flag_time = True
                        flag_ch_2_and_3 = True
                    else:
                        noise_one_d += 1
                        continue
                else:
                    not_recovable_two_d_one_end += 1
                    continue
            elif len(channel_data_tmp[j * 4:j * 4 + upper_side]) == 3:
                if (channel_data_tmp_2[-3] == 0 and channel_data_tmp_2[-2] == 1):
                    time = time_data_tmp_2[0] + time_data_tmp_2[1]
                    time_1 = time * TOFFACTOR2
                    xx_dif = time_data_tmp_2[1] - time_data_tmp_2[0]
                    xx_dif = -0.5 * xx_dif + 1225
                    xx_dif = (((xx_dif - XYBINSHIFT) * XYFACTOR) * 0.1)
                    tof_one_d.append(time_1)
                    v_dc_one_d.append(x['high_voltage'])
                    x_det_one_d.append(xx_dif)
                    y_det_one_d.append(0)
                    v_p__one_d.append(x['pulse'])
                    if abs(xx_dif) <= 4:
                        flag_time = True
                        flag_ch_0_and_1 = True
                    else:
                        noise_one_d += 1
                        continue
                elif (channel_data_tmp_2[-3] == 2 and channel_data_tmp_2[-2] == 3):
                    time = time_data_tmp_2[0] + time_data_tmp_2[1]
                    time_1 = time * TOFFACTOR2
                    xx_dif = time_data_tmp_2[1] - time_data_tmp_2[0]
                    xx_dif = -0.5 * xx_dif + 1225
                    xx_dif = (((xx_dif - XYBINSHIFT) * XYFACTOR) * 0.1)
                    tof_one_d.append(time_1)
                    v_dc_one_d.append(x['high_voltage'])
                    x_det_one_d.append(xx_dif)
                    y_det_one_d.append(0)
                    v_p__one_d.append(x['pulse'])
                    if abs(xx_dif) <= 4:
                        flag_time = True
                        flag_ch_2_and_3 = True
                    else:
                        noise_one_d += 1
                        continue

                elif (channel_data_tmp_2[-2] == 2 and channel_data_tmp_2[-1] == 3):
                    time = time_data_tmp_2[1] + time_data_tmp_2[2]
                    time_1 = time * TOFFACTOR2
                    yy_dif = time_data_tmp_2[2] - time_data_tmp_2[1]
                    yy_dif = -0.5 * yy_dif + 1225
                    yy_dif = (((yy_dif - XYBINSHIFT) * XYFACTOR) * 0.1)
                    tof_one_d.append(time_1)
                    v_dc_one_d.append(x['high_voltage'])
                    x_det_one_d.append(0)
                    y_det_one_d.append(yy_dif)
                    v_p__one_d.append(x['pulse'])
                    if abs(yy_dif) <= 4:
                        flag_time = True
                        flag_ch_2_and_3 = True
                    else:
                        noise_one_d += 1
                        continue
                elif (channel_data_tmp_2[-2] == 0 and channel_data_tmp_2[-1] == 1):
                    time = time_data_tmp_2[1] + time_data_tmp_2[2]
                    time_1 = time * TOFFACTOR2
                    xx_dif = time_data_tmp_2[2] - time_data_tmp_2[1]
                    xx_dif = -0.5 * xx_dif + 1225
                    xx_dif = (((xx_dif - XYBINSHIFT) * XYFACTOR) * 0.1)
                    tof_one_d.append(time_1)
                    v_dc_one_d.append(x['high_voltage'])
                    x_det_one_d.append(xx_dif)
                    y_det_one_d.append(0)
                    v_p__one_d.append(x['pulse'])
                    if abs(xx_dif) <= 4:
                        flag_time = True
                        flag_ch_0_and_1 = True
                    else:
                        noise_one_d += 1
                        continue
                else:
                    not_recovable_out_of_detector += 1
                    continue

            elif len(channel_data_tmp[j * 4:j * 4 + upper_side]) == 4:
                # Initialize variables to store the indices
                index_of_0 = -1
                index_of_1 = -1
                index_of_2 = -1
                index_of_3 = -1

                # Iterate through the array
                for i, num in enumerate(channel_data_tmp_2):
                    # Check for the first occurrence of each number
                    if num == 0 and index_of_0 == -1:
                        index_of_0 = i
                    elif num == 1 and index_of_1 == -1:
                        index_of_1 = i
                    elif num == 2 and index_of_2 == -1:
                        index_of_2 = i
                    elif num == 3 and index_of_3 == -1:
                        index_of_3 = i
                if index_of_0 != -1 and index_of_1 != -1:
                    time = time_data_tmp_2[index_of_0] + time_data_tmp_2[index_of_1]
                    time_1 = time * TOFFACTOR2
                    xx_dif = time_data_tmp_2[index_of_1] - time_data_tmp_2[index_of_0]
                    xx_dif = -0.5 * xx_dif + 1225
                    xx_dif = (((xx_dif - XYBINSHIFT) * XYFACTOR) * 0.1)
                    tof_one_d.append(time_1)
                    v_dc_one_d.append(x['high_voltage'])
                    x_det_one_d.append(xx_dif)
                    y_det_one_d.append(0)
                    v_p__one_d.append(x['pulse'])
                    if abs(xx_dif) <= 4:
                        flag_time = True
                        flag_ch_0_and_1 = True
                    else:
                        noise_one_d += 1
                        continue
                elif index_of_2 != -1 and index_of_3 != -1:
                    time = time_data_tmp_2[index_of_2] + time_data_tmp_2[index_of_3]
                    time_1 = time * TOFFACTOR2
                    yy_dif = time_data_tmp_2[index_of_3] - time_data_tmp_2[index_of_2]
                    yy_dif = -0.5 * yy_dif + 1225
                    yy_dif = (((yy_dif - XYBINSHIFT) * XYFACTOR) * 0.1)
                    tof_one_d.append(time_1)
                    v_dc_one_d.append(x['high_voltage'])
                    x_det_one_d.append(0)
                    y_det_one_d.append(yy_dif)
                    v_p__one_d.append(x['pulse'])
                    if abs(yy_dif) <= 4:
                        flag_time = True
                        flag_ch_2_and_3 = True
                    else:
                        noise_one_d += 1
                        continue
                else:
                    not_recovable_out_of_detector += 1
                    continue

            else:
                print('fault length channel')
                print(channel_data_tmp)
                print(len(channel_data_tmp[j * 4:]))
                print(channel_data_tmp_2)
                print(x['valid_event'])
                print('------------------')
            if time_1 != 0 and flag_time == True:
                if 135 < time_1 < 159:
                    tof_range_1_one_d += 1
                    if flag_ch_0_and_1:
                        num_ch_0_and_1 += 1
                    elif flag_ch_2_and_3:
                        num_ch_2_and_3 += 1
                elif 392 < time_1 < 463:
                    tof_range_2_one_d += 1
                    if flag_ch_0_and_1:
                        num_ch_0_and_1 += 1
                    elif flag_ch_2_and_3:
                        num_ch_2_and_3 += 1
                elif 535 < time_1 < 646:
                    tof_range_3_one_d += 1
                    if flag_ch_0_and_1:
                        num_ch_0_and_1 += 1
                    elif flag_ch_2_and_3:
                        num_ch_2_and_3 += 1
                else:
                    noise_one_d += 1
            else:
                print('fault in time_1 calculation')
                print(channel_data_tmp_2)
            time_1 = 0
            yy_dif = 0
            xx_dif = 0
            flag_time = False
            flag_ch_0_and_1 = False
            flag_ch_2_and_3 = False

In [ ]:
print('sum two d:', tof_range_1 + tof_range_2 + tof_range_3 + noise_two_d, 'length time two d:', len(tof_two_d))
print('sum one d:', tof_range_1_one_d + tof_range_2_one_d + tof_range_3_one_d + noise_one_d, 'length time two d:', len(tof_one_d))

In [ ]:
print('num_ch_0_1:', num_ch_0_and_1, '---- num_ch_2_3:', num_ch_2_and_3)

In [ ]:
ch_0_1 = len([element for element in x_det_one_d if element != 0])
ch_2_3 = len([element for element in y_det_one_d if element != 0])
print('num_ch_0_1:', ch_0_1, '---- num_ch_2_3:', ch_2_3)

In [ ]:
t_one_d = np.array(tof_one_d)
t_two_d = np.array(tof_two_d)
tt_one_d = t_one_d[t_one_d<1000]
tt_two_d = t_two_d[t_two_d<1000]

# Define the bin edges with a bin size of 0.1
bin_size = 0.1
min_edge = min(np.min(tt_one_d), np.min(tt_two_d))
max_edge = max(np.max(tt_one_d), np.max(tt_two_d))
# bins = np.arange(min_edge, max_edge + bin_size, bin_size)
bins = np.linspace(min_edge,max_edge, round(max_edge / bin_size))

# Plot the histograms
fig, ax = plt.subplots(figsize=(7, 2.5))
ax.hist(tt_two_d, bins=bins, color='blue', alpha=0.7, label='4 DLTS', log=True, histtype='stepfilled')
ax.hist(tt_one_d, bins=bins, color='orange', alpha=0.7, label='2 DLTS', log=True, histtype='stepfilled')

# Add titles and labels
plt.xlabel('Time of Flight [ns]')
plt.ylabel('Event Counts')
plt.legend()
plt.tight_layout()
plt.savefig("raw_dlts_tof_overlay1.svg", format="svg", dpi=600)
plt.savefig("raw_dlts_tof_overlay1.png", format="png", dpi=600)

# Show the plot
plt.show()


In [ ]:
len([element for element in y_det_one_d if element != 0])

In [ ]:
t_one_d = np.array(tof_one_d)
t_two_d = np.array(tof_two_d)


mc_one_d = mc_tools.tof2mc(t_one_d, 42, np.array(v_dc_one_d), np.array(x_det_one_d), np.array(y_det_one_d), 110, np.array(v_p__one_d), mode='voltage')
mc_two_d = mc_tools.tof2mc(t_two_d, 42, np.array(v_dc_two_d), np.array(x_det_two_d), np.array(y_det_two_d), 110, np.array(v_p__two_d), mode='voltage')

mc_one_d = mc_one_d[mc_one_d < 40]
mc_two_d = mc_two_d[mc_two_d < 40]

# Define the bin edges with a bin size of 0.1
bin_size = 0.1
min_edge = min(np.min(mc_one_d), np.min(mc_two_d))
max_edge = max(np.max(mc_one_d), np.max(mc_two_d))
# bins = np.arange(min_edge, max_edge + bin_size, bin_size)
bins = np.linspace(min_edge,max_edge, round(max_edge / bin_size))

# Plot the histograms
fig, ax = plt.subplots(figsize=(7, 2.5))
ax.hist(mc_two_d, bins=bins, color='blue', alpha=0.7, label='4 DLTS', log=True, histtype='stepfilled')
ax.hist(mc_one_d, bins=bins, color='orange', alpha=0.7, label='2 DLTS', log=True, histtype='stepfilled')

# Add titles and labels
plt.xlabel('Time of Flight [ns]')
plt.ylabel('Event Counts')
plt.legend()

plt.savefig("raw_dlts_mc_overlay1.svg", format="svg", dpi=600)
plt.savefig("raw_dlts_mc_overlay1.png", format="png", dpi=600)
plt.tight_layout()
# Show the plot
plt.show()


In [ ]:
mask_h_one_d = (mc_one_d > 0.89) & (mc_one_d < 1.1)
mask_al_2plus_one_d = (mc_one_d > 14.3) & (mc_one_d < 16.697)
mask_al_1plus_one_d = (mc_one_d > 28.56) & (mc_one_d < 34.76)

# Define masks for mc_two_d (assuming they are defined similarly to mc_one_d)
mask_h_two_d = (mc_two_d > 0.99) & (mc_two_d < 1.2)
mask_al_2plus_two_d = (mc_two_d > 13.9) & (mc_two_d < 15.4)
mask_al_1plus_two_d = (mc_two_d > 28.1) & (mc_two_d < 31.4)


# Count the number of True values in each mask
num_true_h_one_d = np.sum(mask_h_one_d)
num_true_al_2plus_one_d = np.sum(mask_al_2plus_one_d)
num_true_al_1plus_one_d = np.sum(mask_al_1plus_one_d)

# Calculate the total number of True values across all masks
total_trues_one_d = num_true_h_one_d + num_true_al_2plus_one_d + num_true_al_1plus_one_d


###########################################



# Count the number of True values in each mask
num_true_h_two_d = np.sum(mask_h_two_d)
num_true_al_2plus_two_d = np.sum(mask_al_2plus_two_d)
num_true_al_1plus_two_d = np.sum(mask_al_1plus_two_d)

# Calculate the total number of True values across all masks
total_trues_two_d = num_true_h_two_d + num_true_al_2plus_two_d + num_true_al_1plus_two_d

total_trues = total_trues_two_d + total_trues_one_d
# Calculate the percentage of True values for each mask
percentage_h_one_d = (num_true_h_one_d / total_trues) * 100 if total_trues > 0 else 0
percentage_al_2plus_one_d = (num_true_al_2plus_one_d / total_trues) * 100 if total_trues > 0 else 0
percentage_al_1plus_one_d = (num_true_al_1plus_one_d / total_trues) * 100 if total_trues > 0 else 0

# Print the results
print(f"Number of True values in mask_h_one_d: {num_true_h_one_d}")
print(f"Number of True values in mask_al_2plus_one_d: {num_true_al_2plus_one_d}")
print(f"Number of True values in mask_al_1plus_one_d: {num_true_al_1plus_one_d}")

print(f"Percentage of True values in mask_h_one_d: {percentage_h_one_d:.2f}%")
print(f"Percentage of True values in mask_al_2plus_one_d: {percentage_al_2plus_one_d:.2f}%")
print(f"Percentage of True values in mask_al_1plus_one_d: {percentage_al_1plus_one_d:.2f}%")



# Calculate the percentage of True values for each mask
percentage_h_two_d = (num_true_h_two_d / total_trues) * 100 if total_trues > 0 else 0
percentage_al_2plus_two_d = (num_true_al_2plus_two_d / total_trues) * 100 if total_trues > 0 else 0
percentage_al_1plus_two_d = (num_true_al_1plus_two_d / total_trues) * 100 if total_trues > 0 else 0

# Print the results
print(f"Number of True values in mask_h_two_d: {num_true_h_two_d}")
print(f"Number of True values in mask_al_2plus_two_d: {num_true_al_2plus_two_d}")
print(f"Number of True values in mask_al_1plus_two_d: {num_true_al_1plus_two_d}")

print(f"Percentage of True values in mask_h_two_d: {percentage_h_two_d:.2f}%")
print(f"Percentage of True values in mask_al_2plus_two_d: {percentage_al_2plus_two_d:.2f}%")
print(f"Percentage of True values in mask_al_1plus_two_d: {percentage_al_1plus_two_d:.2f}%")

In [ ]:
# Create labels and corresponding values for the bar chart
labels = ['${}^{1}$H$^+$ 4 DLTS', '${}^{27}$Al$^{2+}$ 4 DLTS', '${}^{27}$Al$^+$ 4 DLTS', 'Noise 4 DLTS',
          '${}^{1}$H$^+$ 2 DLTS', '${}^{27}$Al$^{2+}$ 2 DLTS', '${}^{27}$Al$^+$ 2 DLTS', 'Noise 2 DLTS',
          'Not recoverable 2 different DLTS', 'Not recoverable out of detector']
values = [tof_range_1, tof_range_2, tof_range_3, noise_two_d, tof_range_1_one_d, tof_range_2_one_d, tof_range_3_one_d, noise_one_d, not_recovable_two_d_one_end, not_recovable_out_of_detector]
# Define a list of colors for each bar
colors = ['orange', 'orange', 'orange', 'orange', 'green', 'green', 'green', 'green', 'red', 'red']

fig, ax = plt.subplots(figsize=(7, 4))
# Create bar chart
ax.bar(labels, values, color=colors)

# Set labels and scale
ax.set_ylabel('Counts')
ax.set_xticklabels(labels, rotation=45, ha='right')
ax.set_yscale('log')

# Adjust layout
fig.tight_layout()
from matplotlib import rcParams
rcParams['svg.fonttype'] = 'none'
plt.savefig("raw_dlets_stat1.svg", format="svg", dpi=600)
plt.savefig("raw_dlets_stat1.png", format="png", dpi=600)
# Show the plot
plt.show()

In [ ]:
delta_x = []
delta_y = []
            
for i, x in enumerate(tqdm(result_total)):
    length = len(x['channels'])
    if len(x['valid_event']) > 2:
        if x['valid_event'][0] == True and x['valid_event'][1] == True:
            time_data_tmp = np.array(x['time_data'], dtype=np.int64)
            # First event
            index = 0
            xx_dif = time_data_tmp[index*4 + 1] - time_data_tmp[index*4 + 0]
            yy_dif = time_data_tmp[index*4 + 3] - time_data_tmp[index*4 + 2]
            time = time_data_tmp[index*4 + 0] + time_data_tmp[index*4 + 1] + time_data_tmp[index*4 + 2] + time_data_tmp[index*4 + 3]
            xx_dif = -0.5*xx_dif + 1225
            yy_dif = -0.5*yy_dif + 1225
            time_1 = time * TOFFACTOR
            xx_dif_1 = (((xx_dif - XYBINSHIFT) * XYFACTOR) * 0.1)
            yy_dif_1 = (((yy_dif - XYBINSHIFT) * XYFACTOR) * 0.1)
            # second event
            index = 1
            xx_dif = time_data_tmp[index*4 + 1] - time_data_tmp[index*4 + 0]
            yy_dif = time_data_tmp[index*4 + 3] - time_data_tmp[index*4 + 2]
            time = time_data_tmp[index*4 + 0] + time_data_tmp[index*4 + 1] + time_data_tmp[index*4 + 2] + time_data_tmp[index*4 + 3]
            xx_dif = -0.5*xx_dif + 1225
            yy_dif = -0.5*yy_dif + 1225
            time_2 = time * TOFFACTOR
            xx_dif_2 = (((xx_dif - XYBINSHIFT) * XYFACTOR) * 0.1)
            yy_dif_2 = (((yy_dif - XYBINSHIFT) * XYFACTOR) * 0.1)
            if abs(xx_dif_1) <= 4 and abs(yy_dif_1) <= 4 and abs(xx_dif_2) <= 4 and abs(yy_dif_2) <= 4:
                if 530 <= time_1 <= 650 and 530 <= time_2 <= 650:
                    if abs(time_1 - time_2) < 20:
                        delta_x.append(xx_dif_1 - xx_dif_2)
                        delta_y.append(yy_dif_1 - yy_dif_2)

In [ ]:
print(min(delta_x), min(delta_y))
delta_x = np.array(delta_x)
delta_y = np.array(delta_y)
euclidean_distance = np.sqrt(delta_x**2 + delta_y**2) * 10 # to make it mm
print(min(euclidean_distance), len(euclidean_distance))
fig1, ax1 = plt.subplots(figsize=(2.5,2.5))
bin_size = 1
edges_x = np.arange(np.min(euclidean_distance), np.max(euclidean_distance), bin_size)
plt.hist(euclidean_distance, bins=edges_x, color='blue', alpha=0.7, histtype='step')

plt.xlabel('Separation Distance (mm)', fontsize=10)
plt.ylabel('Count - Double Hits', fontsize=10)
plt.tight_layout()
from matplotlib import rcParams
rcParams['svg.fonttype'] = 'none'
plt.savefig("dead_zone.svg", format="svg", dpi=600)
# plt.savefig("dead_zone_y.png", format="png", dpi=600)
plt.show()

In [ ]:
count = 0
multi_hit = 0
for x in result_total:
    if x['valid_event'].count(True) > 25:
        print(x['valid_event'].count(True), x['start_counter'][0])
    if x['valid_event'].count(True) > 1:
        multi_hit += 1
    for evn in x['valid_event']:
        if evn == True:
            count += 1
            
print(multi_hit)
print(count)

In [ ]:
det_x_total = []
det_y_total = []
tof_total = []
start_counter_total = []
high_voltage_total = []
pulse_total = []
print('length of result_total:', len(result_total))

# indexx = 1000000
for ion in tqdm(result_total, desc="Processing", unit="iteration"):
#     indexx -= 1
#     if indexx == 0:
#         break
    time_data_tmp = np.array(ion['time_data'], dtype=np.int64)
#     if abs(indexx) <= 1000:
#         print(time_data_tmp)
    for index, event in enumerate(ion['valid_event']):
#         if abs(indexx) <= 1000:
#             print(index, ion['valid_event'], event)
        if event:
            xx_dif = time_data_tmp[index*4 + 1] - time_data_tmp[index*4 + 0]
            yy_dif = time_data_tmp[index*4 + 3] - time_data_tmp[index*4 + 2]
            time = time_data_tmp[index*4 + 0] + time_data_tmp[index*4 + 1] + time_data_tmp[index*4 + 2] + time_data_tmp[index*4 + 3]
            xx_dif = -0.5*xx_dif + 1225
            yy_dif = -0.5*yy_dif + 1225
            time = time * TOFFACTOR
            xx_dif = (((xx_dif - XYBINSHIFT) * XYFACTOR) * 0.1)
            yy_dif = (((yy_dif - XYBINSHIFT) * XYFACTOR) * 0.1)
            if abs(xx_dif) <= 4 and abs(yy_dif) <= 4:
                det_x_total.append(xx_dif)
                det_y_total.append(yy_dif)
                tof_total.append(time)
                start_counter_total.append(np.array(ion['start_counter'], dtype=np.int64)[0])
                high_voltage_total.append(ion['high_voltage'])
                pulse_total.append(ion['pulse'])

print(len(det_x_total))

In [ ]:
#dead zone

start_index, end_index, max_count, sc_num = raw_data_surface_concept.find_nth_max_repeated_indices(start_counter_total.copy(), 1)

print(start_index, end_index, max_count, sc_num)
det_x_sc = det_x_total[start_index:end_index+1]
det_y_sc = det_y_total[start_index:end_index+1]


dx = np.subtract.outer(det_x_sc, det_x_sc)
dy = np.subtract.outer(det_y_sc, det_y_sc)

dx = abs(dx)
dy = abs(dy)

dx = dx[dx!=0]
dy = dy[dy!=0]
print('min dx:', np.min(dx))
print('min dy:', np.min(dy))

# Plot histograms of dx and dy
fig1, ax1 = plt.subplots(figsize=(2.5,2.5))
bin_size = 0.1
edges_x = np.arange(np.min(dx), np.max(dx), bin_size)
plt.hist(dx.flatten(), bins=edges_x, color='blue', alpha=0.7, histtype='step')

plt.xlabel(r'$\Delta x$ (cm)', fontsize=10)
plt.ylabel('Count', fontsize=10)
plt.tight_layout()
from matplotlib import rcParams
rcParams['svg.fonttype'] = 'none'
plt.savefig("dead_zone_x.svg", format="svg", dpi=600)
plt.savefig("dead_zone_y.png", format="png", dpi=600)
plt.show()

# Plot histograms of dx and dy
plt.figure(figsize=(10, 5))
bin_size = 0.1
plt.subplot(1, 3, 1)
edges_x = np.arange(np.min(dx), np.max(dx), bin_size)
plt.hist(dx.flatten(), bins=edges_x, color='blue', alpha=0.7, histtype='step')

plt.xlabel(r'$\Delta x$ (cm)', fontsize=10)
plt.ylabel('Count', fontsize=10)

plt.subplot(1, 3, 2)
edges_y = np.arange(np.min(dy), np.max(dy), bin_size)
plt.hist(dy.flatten(), bins=edges_y, color='red', alpha=0.7, histtype='step')
plt.xlabel(r'$\Delta y$ (cm)', fontsize=10)
plt.ylabel('Count', fontsize=10)

plt.subplot(1, 3, 3)
hist2d, x_edges, y_edges = np.histogram2d(dx, dy, bins=[edges_x, edges_y])
extent = [x_edges[0], x_edges[-1], y_edges[0], y_edges[-1]]
plt.imshow(hist2d.T, origin='lower', extent=extent, aspect="auto", cmap='viridis')
plt.colorbar(label='Counts')
plt.xlabel(r'$\Delta x$ (cm)', fontsize=10)
plt.ylabel(r'$\Delta y$ (cm)', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
fdm = {'x_det (cm)': det_x_total, 'y_det (cm)': det_y_total}
fdm = pd.DataFrame(fdm)

plot_crop_fdm(fdm)
# mc = mc_tools.tof2mc(t=np.array(tof_total), t0=0, V=np.array(high_voltage_total), xDet=np.array(det_x_total), yDet=np.array(det_x_total), flightPathLength=110, V_pulse=np.array(pulse_total), mode='voltage')
mc = mc_tools.tof2mc(t=np.array(tof_total), t0=0, V=np.array(high_voltage_total), xDet=np.array(det_x_total), yDet=np.array(det_x_total), flightPathLength=110, V_pulse=np.zeros(len(det_x_total)), mode='voltage')
mc_plot = AptHistPlotter(mc[mc<50])
y_hist, x_hist = mc_plot.plot_histogram()

In [ ]:
# Create a dictionary with the lists
data_total = {
        'high_voltage (V)': high_voltage_total,
        'pulse': pulse_total,
        'start_counter': start_counter_total,
        't (ns)': tof_total,
        'x_det (cm)': det_x_total,
        'y_det (cm)': det_y_total,
}

# save the DataFrame
df_total = pd.DataFrame(data_total)
# df_total.to_hdf('analyzed_raw_data.h5', key='df', mode='w')

In [ ]:

# Create a dictionary with the lists
data_4 = {
        'high_voltage (V)': high_voltage_4,
        'pulse': pulse_4,
        'start_counter': start_counter_4,
        't (ns)': tof_4,
        'x_det (cm)': det_x_4,
        'y_det (cm)': det_y_4,
}

# save the DataFrame
df_4 = pd.DataFrame(data_4)
# df_4.to_hdf('analyzed_raw_data_4.h5', key='df', mode='w')

In [ ]:
df_total

In [ ]:
counter = df_total['start_counter'].to_numpy()
pulse_pi = np.zeros(len(counter))
ion_pp = np.zeros(len(counter))


multi_hit_count = 1

for i, current_counter in enumerate(counter):
    if i == 0:
        pulse_pi[i] = 0
        previous_counter = current_counter
    else:
        sc = current_counter - previous_counter
        if sc < 0:
            sc_a = 20000 - previous_counter
            sc_b = current_counter
            sc = sc_a + sc_b
            
        pulse_pi[i] = sc
    
        if current_counter == previous_counter:
            multi_hit_count += 1
        else:
            for j in range(multi_hit_count):
                if i + j < len(counter):
                    ion_pp[i - j - 1] = multi_hit_count
    
    
            multi_hit_count = 1
            previous_counter = current_counter
df_total['delta_p'] = pulse_pi.astype(np.uintc)
df_total['multi'] = ion_pp.astype(np.uintc)
df_total[:10]

In [ ]:
filtered_df = df_total[df_total['ion_pp'] == 2]
# filtered_df.reset_index(drop=True, inplace=True)
filtered_df

In [ ]:
from pyccapt.calibration.data_tools import data_tools, data_loadcrop

delta_p, ion_pp = data_loadcrop.calculate_ppi_and_ipp(df_total, max_start_counter=20000)